## duration-prediction video steps


In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import pickle 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
source_dir = "data/"

green_jan = pd.read_parquet(source_dir + "green_tripdata_2022-01.parquet")
#green_feb = 
yellow_jan = pd.read_parquet(source_dir + "yellow_tripdata_2022-01.parquet")
#yellow_feb = 
green_jan.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2022-01-01 00:14:21,2022-01-01 00:15:33,N,1.0,42,42,1.0,0.44,3.5,0.50,0.5,0.00,0.0,None,0.3,4.80,2.0,1.0,0.00
1,1,2022-01-01 00:20:55,2022-01-01 00:29:38,N,1.0,116,41,1.0,2.10,9.5,0.50,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.00
2,1,2022-01-01 00:57:02,2022-01-01 01:13:14,N,1.0,41,140,1.0,3.70,14.5,3.25,0.5,4.60,0.0,None,0.3,23.15,1.0,1.0,2.75
3,2,2022-01-01 00:07:42,2022-01-01 00:15:57,N,1.0,181,181,1.0,1.69,8.0,0.50,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2022-01-01 00:07:50,2022-01-01 00:28:52,N,1.0,33,170,1.0,6.26,22.0,0.50,0.5,5.21,0.0,None,0.3,31.26,1.0,1.0,2.75
5,1,2022-01-01 00:47:57,2022-01-01 00:54:09,N,1.0,150,210,1.0,1.30,7.0,0.50,0.5,0.00,0.0,None,0.3,8.30,2.0,1.0,0.00
6,2,2022-01-01 00:13:38,2022-01-01 00:33:50,N,1.0,66,67,1.0,6.47,22.5,0.50,0.5,0.00,0.0,None,0.3,23.80,2.0,1.0,0.00
7,2,2022-01-01 00:43:00,2022-01-01 00:49:20,N,1.0,40,195,1.0,1.15,6.0,0.50,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.00
8,2,2022-01-01 00:41:04,2022-01-01 00:47:04,N,1.0,112,80,1.0,1.30,6.0,0.50,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.00
9,2,2022-01-01 00:51:07,2022-01-01 01:09:31,N,1.0,256,186,1.0,4.75,17.0,0.50,0.5,4.21,0.0,None,0.3,25.26,1.0,1.0,2.75


In [3]:
print(yellow_jan.shape)
yellow_jan.head(10)


(2463931, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [4]:
def load_and_preprocess_duration_data(filename, type = "green"):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        if type == "green":
            df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
            df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        else:
            df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
            df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    if type == "green":
        df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    else:
        df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    #change to minutes
    df.duration = df.duration.apply(lambda timedelta: timedelta.total_seconds() / 60)
    # focus on rides under one hour
    #df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = load_and_preprocess_duration_data(source_dir +"yellow_tripdata_2022-01.parquet", type = "yellow")
df_val = load_and_preprocess_duration_data(source_dir + "yellow_tripdata_2022-02.parquet",type = "yellow")

In [6]:
# Q2 jan sd duration
df_train["duration"].std() 

46.44530513776802

In [23]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)] 

In [7]:
# Q3 fraction of remaining records
len(df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]) / len(df_train)


0.9827547930522406

In [8]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [24]:
dv = DictVectorizer()

categorical = ['PULocationID', 'DOLocationID']


train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_train.shape

(2421440, 515)

In [25]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val.shape

(2918187, 515)

In [26]:

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred_train = lr.predict(X_train)

mean_squared_error(y_train, y_pred_train, squared=False)

6.986190135971508

In [28]:
df_val

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val.shape

(2918187, 515)

In [29]:

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7863894855242926